In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import QUANTAXIS as QA
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

crsp fiscal resample

In [4]:
def calc_exf(df):
    ret = df.ret_p1
    dret = df.dret_p1
    exf = df.mv - df.mv.shift(1) * df.retx_p1
    exf_to_mv = exf / df.mv.shift(1)
    
    car1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
    car3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
    car5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    wr5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) / \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    ret_n1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n2 = ((ret[::-1]).rolling(2, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n4 = ((ret[::-1]).rolling(4, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n6 = ((ret[::-1]).rolling(6, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n7 = ((ret[::-1]).rolling(7, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n8 = ((ret[::-1]).rolling(8, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    res = pd.DataFrame({
        'date':df.date,
        'mv':df.mv,
        'retx_p1':df.retx_p1,
        'exf':exf,
        'exf_to_mv':exf_to_mv,
        'car1':car1.shift(-1),
        'car3':car3.shift(-1),
        'car5':car5.shift(-1),
        'wr5':wr5.shift(-1),
        'ret_n1':ret_n1.shift(-1),
        'ret_n2':ret_n2.shift(-1),
        'ret_n3':ret_n3.shift(-1),
        'ret_n4':ret_n4.shift(-1),
        'ret_n5':ret_n5.shift(-1),
        'ret_n6':ret_n6.shift(-1),
        'ret_n7':ret_n7.shift(-1),
        'ret_n8':ret_n8.shift(-1)
    })
    return res

In [5]:
crsp_fa = pd.read_csv('data/crsp_fa.zip',
                     parse_dates=['date'], infer_datetime_format=True)

查看firm-years数量

ps:因为流通股数的单位默认是1000, 故mv_adj只需要大于50000即可

In [6]:
crsp_fa[crsp_fa.mv_adj >= 5e+4].shape[0]

145087

In [7]:
crsp_fa = crsp_fa[crsp_fa.mv_adj >= 5e+4]

In [8]:
tic = time.perf_counter()
crsp_fa = crsp_fa.set_index('permno').groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

262.666677



exf_to_mv : $EXF_{t}/MV_{t-1}$

ret_n1 : 接下来1年的return


In [10]:
from scipy.stats import mstats

In [20]:
crsp_fa.head()

,date,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
permno,,,,,,,,,,,,,,,,,
10002,1993-12-31,38612.125,1.430557,NaN,NaN,0.111624,-0.337573,0.341395,0.341395,1.039473,1.135239,1.085200,2.130205,2.027591,1.663451,1.310096,1.466758
10002,1994-12-31,39361.875,1.019418,-0.008577,-2.221426e-07,-0.281362,0.121447,-0.897720,-0.897720,1.092129,1.043990,2.049311,1.950594,1.600282,1.260346,1.411059,1.653241
10002,1995-12-31,42028.000,1.066668,41.937460,1.065433e-03,-0.160509,0.462912,-0.522323,-0.522323,0.955922,1.876438,1.786048,1.465287,1.154027,1.292026,1.513779,2.259641
10002,1996-12-31,39026.000,0.928571,-0.001807,-4.298500e-08,0.705720,-0.096200,-0.527141,-0.527141,1.962960,1.868403,1.532852,1.207239,1.351602,1.583579,2.363833,3.730554
10002,1997-12-31,104027.000,1.884614,30478.045526,7.809677e-01,0.009172,-0.579293,-0.533391,-0.533391,0.951829,0.780888,0.615009,0.688553,0.806730,1.204218,1.900473,1.593407


winsorize

In [145]:
crsp_fa1 = crsp_fa.copy()
crsp_fa1 = crsp_fa1.dropna()
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.05, 0.05])

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
exf_to_mv,,,,,,,,,,,,,
"(-1.118, -0.0418]",-0.093840,0.142903,0.361576,0.551920,1.322326,1.281421,1.509475,1.765516,2.001629,2.214963,2.425102,2.605698,2.799185
"(-0.0418, -0.00558]",-0.019767,0.051951,0.154975,0.251681,1.169730,1.187597,1.352048,1.525883,1.693877,1.878278,2.061083,2.245242,2.369358
"(-0.00558, -5.22e-08]",-0.001081,0.050358,0.148400,0.241659,1.155783,1.218821,1.426558,1.631001,1.850925,2.095357,2.303060,2.524635,2.735010
"(-5.22e-08, 0.000962]",0.000195,0.060339,0.159756,0.268823,1.168142,1.237140,1.464005,1.675084,1.923029,2.183523,2.417694,2.660599,2.925948
"(0.000962, 0.0048]",0.002707,0.069020,0.157587,0.251900,1.150907,1.228667,1.429806,1.609575,1.792268,2.014996,2.208689,2.421647,2.611440
"(0.0048, 0.0114]",0.007774,0.060625,0.150056,0.242438,1.147202,1.209027,1.405493,1.570037,1.748272,1.943134,2.106602,2.259610,2.421004
"(0.0114, 0.0251]",0.017340,0.050174,0.162521,0.248917,1.150359,1.190564,1.390496,1.563358,1.721234,1.901651,2.047273,2.195383,2.369293
"(0.0251, 0.0635]",0.040462,0.059055,0.153652,0.188842,1.116984,1.193349,1.390052,1.545866,1.693119,1.821442,1.980955,2.109358,2.251230
"(0.0635, 0.192]",0.115142,0.027049,0.082281,0.128894,1.079203,1.164900,1.328075,1.488669,1.634101,1.795988,1.920576,2.084377,2.258798


Significant issuance & Significant repurchase

EXF is greater than 5% of pre-issuance market capitalization

$EXF_t >= 0.05 * MV_{t-1}$

equals

$EXF_t / MV_{t-1} >= 0.05$

In [149]:
crsp_fa1[crsp_fa1.exf_to_mv > 0.05].describe().loc[['count', 'mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
count,2.530200e+04,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000,25302.000000
mean,1.626440e+06,1.395831,284328.609102,0.208905,0.017778,0.037729,0.072488,1.042054,1.145908,1.291543,1.424856,1.569955,1.702793,1.824572,1.963035,2.116956


In [147]:
crsp_fa1[crsp_fa1.exf_to_mv <= -0.05].describe().loc[['count', 'mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
count,1.013200e+04,10132.000000,10132.000000,10132.00000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000,10132.000000
mean,1.842454e+06,1.641603,-257892.482524,-0.09959,0.150803,0.380099,0.581778,1.335757,1.290799,1.528833,1.792418,2.037703,2.255066,2.472703,2.644147,2.849854


In [148]:
crsp_fa1[(crsp_fa1.exf_to_mv > -0.05) & \
         (crsp_fa1.exf_to_mv <= 0.05)].describe().loc[['count', 'mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
count,7.798600e+04,77986.00000,77986.000000,77986.000000,77986.000000,77986.000000,77986.000000,77986.000000,77986.000000,77986.000000,77986.00000,77986.000000,77986.000000,77986.000000,77986.000000,77986.000000
mean,1.952909e+06,1.08014,-2869.243264,0.004326,0.057706,0.156841,0.246312,1.154463,1.209734,1.408842,1.58998,1.776883,1.981659,2.166762,2.353928,2.534514


long Significant repurchase,  short Significant issuance

In [152]:
crsp_fa1[crsp_fa1.exf_to_mv <= -0.05].describe().loc[['mean']] - \
crsp_fa1[crsp_fa1.exf_to_mv > 0.05].describe().loc[['mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
mean,216014.499817,0.245773,-542221.091626,-0.308495,0.133025,0.34237,0.50929,0.293703,0.144891,0.23729,0.367562,0.467748,0.552273,0.648131,0.681112,0.732898


查看样本的个数

In [68]:
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).count().exf_to_mv

exf_to_mv
(-1.126, -0.0418]        11342
(-0.0418, -0.00558]      11342
(-0.00558, -5.23e-08]    11342
(-5.23e-08, 0.000962]    11342
(0.000962, 0.0048]       11342
(0.0048, 0.0114]         11341
(0.0114, 0.0251]         11342
(0.0251, 0.0635]         11342
(0.0635, 0.192]          11342
(0.192, 0.281]           11342
Name: exf_to_mv, dtype: int64

In [163]:
result = crsp_fa.copy()

In [167]:
writer = pd.ExcelWriter('data/sample_fiscal.xlsx', datetime_format='YYYY-MM-DD')
crsp_fa[crsp_fa.permno==77418].to_excel(writer, index=False)
writer.save()